In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 1min 45s, sys: 2.41 s, total: 1min 48s
Wall time: 1min 53s


In [6]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
# "ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [7]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
130423,ICD10CM:A00,Cholera,skos:exactMatch,MONDO:0015766,cholera,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,cholera
130430,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,typhoid fever
130431,ICD10CM:A01.0,Typhoid fever,skos:exactMatch,MONDO:0005619,typhoid fever,semapv:LexicalMatching,oaklib,0.849779,rdfs:label,rdfs:label,typhoid fever
130436,ICD10CM:A01.1,Paratyphoid fever A,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,NaN,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever a
130439,ICD10CM:A01.2,Paratyphoid fever B,skos:closeMatch,MONDO:0018626,paratyphoid fever,semapv:LexicalMatching,oaklib,NaN,rdfs:label,oio:hasRelatedSynonym,paratyphoid fever b


In [8]:
duplicate_df_lex = msdf_lex.df.copy()
duplicate_df_mondo = msdf_mondo.df.head().copy()

duplicate_df_lex = duplicate_df_lex.rename(columns={"subject_id": "object_id","object_id": "subject_id" })
duplicate_df_mondo = duplicate_df_mondo.rename(columns={"subject_id": "object_id","object_id": "subject_id" })

desired_col_sequnce = ["subject_id","subject_label", "predicate_id", "object_id", "object_label", "mapping_justification"]
duplicate_df_lex = duplicate_df_lex[desired_col_sequnce]
duplicate_df_mondo = duplicate_df_mondo[desired_col_sequnce]


msdf_lex.df = pd.concat([duplicate_df_lex, msdf_lex.df]).drop_duplicates()
msdf_mondo.df = pd.concat([duplicate_df_mondo, msdf_mondo.df]).drop_duplicates()

msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification
51,ICD10CM:E30.1,precocious puberty,skos:exactMatch,MONDO:0000088,Precocious puberty,Unspecified
111,ICD10CM:Q20.3,transposition of the great arteries,skos:broadMatch,MONDO:0000153,Discordant ventriculoarterial connection,Unspecified
113,ICD10CM:Q20.5,transposition of the great arteries,skos:broadMatch,MONDO:0000153,Discordant atrioventricular connection,Unspecified
202,ICD10CM:E83,mineral metabolism disease,skos:exactMatch,MONDO:0000226,Disorders of mineral metabolism,Unspecified
222,ICD10CM:A26,erysipeloid,skos:exactMatch,MONDO:0000237,Erysipeloid,Unspecified


In [9]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
62811,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.8,Other reduction defects of upper limb,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53888,MONDO:0016444,primary anetoderma,skos:broadMatch,ICD10CM:L90.1,Anetoderma of Schweninger-Buzzi,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
60691,MONDO:0019050,inherited hemoglobinopathy,skos:broadMatch,ICD10CM:D56.8,Other thalassemias,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62820,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q72.5,Longitudinal reduction defect of tibia,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2401,MONDO:0001195,spotted fever,skos:broadMatch,ICD10CM:A77.0,Spotted fever due to Rickettsia rickettsii,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [10]:
comparison_df[comparison_df['object_id'] == "ICD10CM:D35.2"]

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
65999,MONDO:0021439,benign neoplasm of pituitary gland,skos:exactMatch,ICD10CM:D35.2,Benign neoplasm of pituitary gland,Unspecified,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN
132721,MONDO:0021439,Benign neoplasm of pituitary gland,skos:exactMatch,ICD10CM:D35.2,benign neoplasm of pituitary gland,semapv:LexicalMatching,COMMON_TO_BOTH,NaN,NaN,NaN,NaN,NaN


In [11]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
62811,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.8,Other reduction defects of upper limb,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53888,MONDO:0016444,primary anetoderma,skos:broadMatch,ICD10CM:L90.1,Anetoderma of Schweninger-Buzzi,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
60691,MONDO:0019050,inherited hemoglobinopathy,skos:broadMatch,ICD10CM:D56.8,Other thalassemias,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62820,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q72.5,Longitudinal reduction defect of tibia,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2401,MONDO:0001195,spotted fever,skos:broadMatch,ICD10CM:A77.0,Spotted fever due to Rickettsia rickettsii,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [12]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
136703,ICD10CM:I85,Esophageal varices,skos:exactMatch,MONDO:0001221,esophageal varices,semapv:LexicalMatching,LEXMATCH,oaklib,0.800000,rdfs:label,oio:hasExactSynonym,esophageal varices
136704,ICD10CM:I85,Esophageal varices,skos:exactMatch,MONDO:0001221,esophageal varices,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,esophageal varices
140766,ICD10CM:Q00.2,Iniencephaly,skos:exactMatch,MONDO:0018968,iniencephaly,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,iniencephaly
132314,ICD10CM:C82,Follicular lymphoma,skos:exactMatch,MONDO:0018906,follicular lymphoma,semapv:LexicalMatching,LEXMATCH,oaklib,0.849779,rdfs:label,rdfs:label,follicular lymphoma
132314,MONDO:0018906,Follicular lymphoma,skos:exactMatch,ICD10CM:C82,follicular lymphoma,semapv:LexicalMatching,LEXMATCH,NaN,NaN,NaN,NaN,NaN


In [13]:
# Inspect why these are missing from SSSOM mappings

unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
# Add distances
# Add Levenshtein distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# Add Jaccard Index [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_15955/653029316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,levenshtein_dist,jaccard_index,monge_elkan
4598,MONDO:0001834,visual pathway disorder,skos:exactMatch,ICD10CM:H47.9,Unspecified disorder of visual pathways,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,32,0.410256,1.956522
14570,MONDO:0005079,polyp,skos:exactMatch,ICD10CM:N84,Polyp of female genital tract,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,24,0.827586,1.800000
61343,MONDO:0019228,inborn disorder of histidine metabolism,skos:exactMatch,ICD10CM:E70.40,"Disorders of histidine metabolism, unspecified",Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,21,0.300000,1.974359
71420,MONDO:0056798,disorder of appendix,skos:exactMatch,ICD10CM:K35-K38,Diseases of appendix (K35-K38),Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,14,0.529412,1.955000
6213,MONDO:0002333,splenic abscess,skos:exactMatch,ICD10CM:D73.3,Abscess of spleen,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,14,0.315789,1.937778


In [14]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
62811,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q71.8,Other reduction defects of upper limb,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
53888,MONDO:0016444,primary anetoderma,skos:broadMatch,ICD10CM:L90.1,Anetoderma of Schweninger-Buzzi,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
60691,MONDO:0019050,inherited hemoglobinopathy,skos:broadMatch,ICD10CM:D56.8,Other thalassemias,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
62820,MONDO:0019713,non-syndromic limb reduction defect,skos:broadMatch,ICD10CM:Q72.5,Longitudinal reduction defect of tibia,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
2401,MONDO:0001195,spotted fever,skos:broadMatch,ICD10CM:A77.0,Spotted fever due to Rickettsia rickettsii,Unspecified,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [15]:
%%time
df_dict = split_dataframe(combined_msdf)

/opt/anaconda3/envs/mondo-ingest/lib/python3.9/site-packages/sssom/parsers.py:864: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  subject_prefixes = set(msdf.df[SUBJECT_ID].str.split(":", 1, expand=True)[0])
/opt/anaconda3/envs/mondo-ingest/lib/python3.9/site-packages/sssom/parsers.py:865: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  object_prefixes = set(msdf.df[OBJECT_ID].str.split(":", 1, expand=True)[0])


CPU times: user 6.16 s, sys: 40.8 ms, total: 6.21 s
Wall time: 9.6 s


In [16]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['icd10cm_closematch_mondo',
 'icd10cm_narrowmatch_mondo',
 'icd10cm_broadmatch_mondo',
 'icd10cm_exactmatch_mondo',
 'mondo_closematch_icd10cm',
 'mondo_narrowmatch_icd10cm',
 'mondo_broadmatch_icd10cm',
 'mondo_exactmatch_icd10cm']

In [17]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [18]:
df_dict['mondo_exactmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment
0,MONDO:0000022,Nocturnal enuresis,skos:exactMatch,ICD10CM:N39.44,nocturnal enuresis,semapv:LexicalMatching,LEXMATCH
1,MONDO:0000078,Congenital malformation syndromes predominantl...,skos:exactMatch,ICD10CM:Q87.0,acrocephalopolysyndactyly,semapv:LexicalMatching,LEXMATCH
2,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Precocious puberty,Unspecified,COMMON_TO_BOTH
3,MONDO:0000088,Precocious puberty,skos:exactMatch,ICD10CM:E30.1,precocious puberty,semapv:LexicalMatching,COMMON_TO_BOTH
4,MONDO:0000129,Disorders of lysine and hydroxylysine metabolism,skos:exactMatch,ICD10CM:E72.3,glutaric aciduria,semapv:RegularExpressionReplacement,LEXMATCH
...,...,...,...,...,...,...,...
3928,MONDO:0800026,Congenital central alveolar hypoventilation sy...,skos:exactMatch,ICD10CM:G47.35,"central hypoventilation syndrome, congenital, ...",semapv:LexicalMatching,LEXMATCH
3929,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Idiopathic pulmonary fibrosis,Unspecified,COMMON_TO_BOTH
3930,MONDO:0800029,Idiopathic pulmonary fibrosis,skos:exactMatch,ICD10CM:J84.112,interstitial lung disease 2,semapv:LexicalMatching,COMMON_TO_BOTH
3931,MONDO:8000010,antiphospholipid syndrome,skos:exactMatch,ICD10CM:D68.61,Antiphospholipid syndrome,Unspecified,COMMON_TO_BOTH
